In [1]:
import numpy as np
import sympy as sym
import json
from pydae.urisi.urisi_builder import urisi
import pydae.build_cffi as db
from pydae.build_v2 import builder


In [2]:
grid = urisi('acdc_7bus.hjson')
grid.uz_jacs = True
grid.construct('acdc_7bus')
# grid.compile('acdc_7bus')

-0.5235987755982988


In [3]:
b = db.builder(grid.sys_dict)
b.build()

In [15]:
b = builder(grid.sys_dict)
b.sparse = False
b.mkl = False
b.uz_jacs = True
b.dict2system()
b.functions()
b.jacobians()
b.cwrite()
b.template()
b.compile()


build_v2.py, line 137  g Matrix([[0.0392156862745098*V_A0_0_i + 0.00980392156862745*V_A0_0_r - 0.0196078431372549*V_A0_1_i - 0.00490196078431373*V_A0_1_r - 0.0196078431372549*V_A0_2_i - 0.00490196078431373*V_A0_2_r - 1.69808902702831*V_A1_0_i - 0.424522256757078*V_A1_0_r + 1.69808902702831*V_A1_1_i + 0.424522256757078*V_A1_1_r - i_vsc_A0_a_r, 0.00980392156862745*V_A0_0_i - 0.0392156862745098*V_A0_0_r - 0.00490196078431373*V_A0_1_i + 0.0196078431372549*V_A0_1_r - 0.00490196078431373*V_A0_2_i + 0.0196078431372549*V_A0_2_r - 0.424522256757078*V_A1_0_i + 1.69808902702831*V_A1_0_r + 0.424522256757078*V_A1_1_i - 1.69808902702831*V_A1_1_r - i_vsc_A0_a_i, -0.0196078431372549*V_A0_0_i - 0.00490196078431373*V_A0_0_r + 0.0392156862745098*V_A0_1_i + 0.00980392156862745*V_A0_1_r - 0.0196078431372549*V_A0_2_i - 0.00490196078431373*V_A0_2_r - 1.69808902702831*V_A1_1_i - 0.424522256757078*V_A1_1_r + 1.69808902702831*V_A1_2_i + 0.424522256757078*V_A1_2_r - i_vsc_A0_b_r, -0.00490196078431373*V_A0_0_i + 

In [12]:
U = 400
S = 100e3
I = S/(np.sqrt(3)*U)
P_cond = 0.02*S
C_loss = P_cond/(I**2)/3
print(f'C_loss = {C_loss}')
P_sw = 0.01*S
A_loss = P_sw/3
print(f'A_loss = {A_loss}')


C_loss = 0.031999999999999994
A_loss = 333.3333333333333


In [6]:
I**2*0.03

625.0

In [7]:
grid = urisi('acdc_7bus_s.json')
grid.uz_jacs = False
grid.construct('acdc_7bus_s')
#grid.verbose = True
grid.compile_mkl('acdc_7bus_s')

FileNotFoundError: [Errno 2] No such file or directory: 'acdc_7bus_s.json'

In [ ]:
grid = urisi('acdc_7bus.json')
grid.construct('acdc_7bus_c')
#grid.verbose = True
#grid.construct('acdc_7bus')

vscs_end = [{'bus_ac':'A4','bus_dc':'D4'}]

for vsc in vscs_end:

    bus_ac = vsc['bus_ac']
    bus_dc = vsc['bus_dc']

    V_0_r,V_0_i = sym.symbols(f'V_{bus_ac}_0_r,V_{bus_ac}_0_i', real = True)
    V_1_r,V_1_i = sym.symbols(f'V_{bus_ac}_1_r,V_{bus_ac}_1_i', real = True)
    V_2_r,V_2_i = sym.symbols(f'V_{bus_ac}_2_r,V_{bus_ac}_2_i', real = True)
    V_3_r,V_3_i = sym.symbols(f'V_{bus_ac}_3_r,V_{bus_ac}_3_i', real = True)

    V_dc_0_r,V_dc_0_i = sym.symbols(f'V_{bus_dc}_0_r,V_{bus_dc}_0_i', real = True)
    V_dc_1_r,V_dc_1_i = sym.symbols(f'V_{bus_dc}_1_r,V_{bus_dc}_1_i', real = True)


    V_an = sym.sqrt((V_0_r-V_3_r)**2 + (V_0_i-V_3_i)**2)
    V_bn = sym.sqrt((V_1_r-V_3_r)**2 + (V_1_i-V_3_i)**2)
    V_cn = sym.sqrt((V_2_r-V_3_r)**2 + (V_2_i-V_3_i)**2)

    V_dc = sym.sqrt((V_dc_0_r-V_dc_1_r)**2 + (V_dc_0_i-V_dc_1_i)**2)

    p_vsc_a,p_vsc_b,p_vsc_c = sym.symbols(f'p_vsc_a_{bus_ac},p_vsc_b_{bus_ac},p_vsc_c_{bus_ac}',real=True)
    p_vsc_a,p_vsc_b,p_vsc_c = sym.symbols(f'p_vsc_a_{bus_ac},p_vsc_b_{bus_ac},p_vsc_c_{bus_ac}',real=True)

    K_droop,T_droop = sym.symbols(f'K_droop_{bus_ac},T_droop_{bus_ac}',real=True)

    dp_vsc_a = 1/T_droop*(-p_vsc_a - K_droop*(V_an/231 - V_dc/800))
    dp_vsc_b = 1/T_droop*(-p_vsc_b - K_droop*(V_bn/231 - V_dc/800))
    dp_vsc_c = 1/T_droop*(-p_vsc_c - K_droop*(V_cn/231 - V_dc/800))

    grid.dae['f'] += [dp_vsc_a]
    grid.dae['f'] += [dp_vsc_b]
    grid.dae['f'] += [dp_vsc_c]

    grid.dae['x'] += [p_vsc_a]
    grid.dae['x'] += [p_vsc_b]
    grid.dae['x'] += [p_vsc_c]

    grid.dae['u_ini_dict'].pop(f'p_vsc_a_{bus_ac}')
    grid.dae['u_ini_dict'].pop(f'p_vsc_b_{bus_ac}')
    grid.dae['u_ini_dict'].pop(f'p_vsc_c_{bus_ac}')

    grid.dae['u_run_dict'].pop(f'p_vsc_a_{bus_ac}')
    grid.dae['u_run_dict'].pop(f'p_vsc_b_{bus_ac}')
    grid.dae['u_run_dict'].pop(f'p_vsc_c_{bus_ac}')

    grid.dae['params_dict'].update({f'K_droop_{bus_ac}':0.0,f'T_droop_{bus_ac}':1.0})

grid.compile_mkl('acdc_7bus_c')

2023-07-03 22:45:01,393 check_system
2023-07-03 22:45:01,411 computing jacobians Fx_run,Fy_run
2023-07-03 22:45:01,530 computing jacobians Gx_run,Gy_run
2023-07-03 22:45:03,091 computing jacobians Fu_run,Gu_run
2023-07-03 22:45:03,396 computing jacobians Fx_ini,Fy_ini
2023-07-03 22:45:03,483 computing jacobians Gy_ini
2023-07-03 22:45:05,571 computing jacobians Hx_run,Hy_run,Hu_run
2023-07-03 22:45:07,263 end checking and computing jacobians
2023-07-03 22:45:07,266 f_ini symbolic to c and xyup
2023-07-03 22:45:07,304 end f_ini symbolic to c and xyup
2023-07-03 22:45:07,306 f_run symbolic to c and xyup
2023-07-03 22:45:07,329 end f symbolic to c and xyup
2023-07-03 22:45:07,330 g_ini symbolic to c and xyup
2023-07-03 22:45:07,781 end g_ini symbolic to c and xyup
2023-07-03 22:45:07,783 g_run symbolic to c and xyup
2023-07-03 22:45:08,215 end g_run symbolic to c and xyup
2023-07-03 22:45:08,218 h symbolic to c and xyup
2023-07-03 22:45:08,377 end h symbolic to c and xyup
2023-07-03 22:45

In [ ]:
import hjson

In [ ]:
def change_line(model,bus_j,bus_k,data_line_code,length,N_branches):
    """
    Change line parameters.

    If the argument 'additional' is passed, then it is appended after the main info.

    Parameters
    ----------
    model : pydae object
            pydae model of the grid where the line will be modified
    bus_j : str
            name of the from bus of the line
    bus_k : str
            name of the to bus of the line
    data_line_code : dict 
            dictionary with 2 keys 'R' and 'X' and the respective matrices as list of lists (see example below)

            
    Returns
    -------
    None


    Example
    -------

    data_line_code = {"R":[[ 0.211,  0.049,  0.049,  0.049],
                           [ 0.049,  0.211,  0.049,  0.049],
                           [ 0.049,  0.049,  0.211,  0.049],
                           [ 0.049,  0.049,  0.049,  0.211]],
                      "X":[[ 0.747,  0.673,  0.651,  0.673],
                           [ 0.673,  0.747,  0.673,  0.651],
                           [ 0.651,  0.673,  0.747,  0.673],
                           [ 0.673,  0.651,  0.673,  0.747]], "I_max":430.0}
    
    length = 10
    N_branches = 4
    change_line(model,'A2','A3',data_line_code,length,N_branches)
    
    """
    

    R_primitive_matrix = np.array(data_line_code['R'])
    X_primitive_matrix = np.array(data_line_code['X'])
    Z_primitive_matrix = R_primitive_matrix + 1j*X_primitive_matrix
    Y_primitive_matrix = np.linalg.inv(Z_primitive_matrix*length/1000)

    for ibranch in range(N_branches):

        node_j = ibranch
        node_k = ibranch    

        for icol in range(N_branches):
            line_name = f"{bus_j}_{node_j}_{bus_k}_{node_k}_{icol}"
            g_jk  = f"g_{line_name}" 
            b_jk  = f"b_{line_name}" 
            bs_jk = f"bs_{line_name}"
                           
            model.set_value(g_jk,Y_primitive_matrix[ibranch,icol].real)
            model.set_value(b_jk,Y_primitive_matrix[ibranch,icol].imag)

    return None

In [ ]:
data_line_code = {"R":[[ 0.211,  0.049,  0.049,  0.049],
                       [ 0.049,  0.211,  0.049,  0.049],
                       [ 0.049,  0.049,  0.211,  0.049],
                       [ 0.049,  0.049,  0.049,  0.211]],
                  "X":[[ 0.747,  0.673,  0.651,  0.673],
                       [ 0.673,  0.747,  0.673,  0.651],
                       [ 0.651,  0.673,  0.747,  0.673],
                       [ 0.673,  0.651,  0.673,  0.747]], "I_max":430.0}

import acdc_7bus_c
model = acdc_7bus_c.model()
length = 10
N_branches = 4
change_line(model,'A2','A3',data_line_code,length,N_branches)

NameError: name 'jacs' is not defined

In [ ]:
change_line?

Signature: change_line(model, bus_j, bus_k, data_line_code, length, N_branches)
Docstring:
Change line parameters.

If the argument 'additional' is passed, then it is appended after the main info.

Parameters
----------
model : pydae object
        pydae model of the grid where the line will be modified
bus_j : str
        name of the from bus of the line
bus_k : str
        name of the to bus of the line
data_line_code : dict 
        dictionary with 2 keys 'R' and 'X' and the respective matrices as list of lists (see example below)

        
Returns
-------
None


Example
-------

data_line_code = {"R":[[ 0.211,  0.049,  0.049,  0.049],
                [ 0.049,  0.211,  0.049,  0.049],
                [ 0.049,  0.049,  0.211,  0.049],
                [ 0.049,  0.049,  0.049,  0.211]],
        "X":[[ 0.747,  0.673,  0.651,  0.673],
                [ 0.673,  0.747,  0.673,  0.651],
                [ 0.651,  0.673,  0.747,  0.673],
                [ 0.673,  0.651,  0.673,  0.747]], "I_m

In [ ]:
11/15*10

7.333333333333333

In [ ]:
0.3*